# Preprocessing 

 tlc data
### feature selection

In [1]:
import pandas as pd
import pyspark.sql.functions as F
import warnings
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col, unix_timestamp, date_format, when, weekofyear, desc, round, expr, hour
from pyspark.sql.types import DateType
from pyspark.ml.feature import StringIndexer

# Suppress warnings
warnings.filterwarnings('ignore')



In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

23/08/20 17:59:00 WARN Utils: Your hostname, LAPTOP-D9335T9D resolves to a loopback address: 127.0.1.1; using 192.168.0.77 instead (on interface wifi0)
23/08/20 17:59:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/20 17:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Preprocess the taxi data and save into raw data file 

### Removing unneccessary features of Taxi data 
### saving the new data in the raw data file 

#### Exmaine the feature Congestion_Surcharge(CS).

In [3]:
count_CS = 0
total = 0
# interate through each taxi data monthly
for i in ['01','02','03','04','05','06']:
    taxi_sdf = spark.read.parquet('../data/landing/tlc_data/'+'yellow_tripdata_2018-'+i+'.parquet')
    # select the congestion feature
    surcharge_data = taxi_sdf.select("Congestion_Surcharge")
    # filter out null values
    sdf_without_nulls = surcharge_data.na.drop()
    #counting 
    count_CS += sdf_without_nulls.count()
    total += taxi_sdf.count()


print('number of non empty records: %d', count_CS)
print('total number of record: %d ', total)
print(count_CS/total)


number of non empty records: %d 14
total number of record: %d  53930466
2.595935291936843e-07


Since only 14 instance of data with non empty Congestion surcharge. We will not consider in analysis. 

#### Examine the MTA_tax 

By TLC dictionary, MTA_tax should only be $0.5

In [4]:
# we randomly examine yellow taxi data in Jan
taxi_sdf = spark.read.parquet('../data/landing/tlc_data/'+'yellow_tripdata_2018-01.parquet')
df = taxi_sdf.select("MTA_tax")
counts = df[['MTA_tax']] \
                .groupby('MTA_tax') \
                .agg(
                    {
                        'MTA_tax': 'count' # count number of instances from sample
                    }
                )

counts

MTA_tax,count(MTA_tax)
45.49,1
0.0,35938
23.8,1
0.6,1
0.35,4
0.5,8720505
0.32,1
0.4,1
3.0,73
6.33,1


As we can see there are some outlier within the data, we will ensure clean these invalid data before analysis.   

#### Examine Airport_fee attribute

In [5]:
count_not_na = 0
total = 0
for i in ['01','02','03','04','05','06']:
    taxi_sdf = spark.read.parquet('../data/landing/tlc_data/'+'yellow_tripdata_2018-'+i+'.parquet')
    sdf = taxi_sdf.select("Airport_fee")
    sdf_without_nulls = sdf.na.drop()
    count_not_na += sdf_without_nulls.count()
    total += taxi_sdf.count()

print(count_not_na)
print(total)
print(count_not_na/total)



12
53930466
2.225087393088723e-07


Similarly, we see only 12 record with non null vaule for airport_fee hence we will not consider this feature in our further investigation 

#### Examine payment type 

In [6]:
taxi_sdf = spark.read.parquet('../data/landing/tlc_data/*')
df = taxi_sdf.select("payment_type")
counts = df[['payment_type']] \
                .groupby('payment_type') \
                .agg(
                    {
                        'payment_type': 'count' # count number of instances from sample
                    }
                )

counts

payment_type,count(payment_type)
1,37571710
3,285325
2,15996221
4,77210


Similarly, we see only 12 record with non null vaule for airport_fee hence we will not consider this feature in our further investigation 

## Filter the original tlc taxi data and save to raw data file
Ensure consistent data type for each feature

In [7]:
# saving the taxi data with only relevant features 

# Define the data type conversions for each column
data_type_mappings = {
    'tpep_pickup_datetime': 'timestamp',
    'tpep_dropoff_datetime': 'timestamp',
    'passenger_count': 'int',
    'trip_distance': 'double',
    'PULocationID': 'int',
    'DOLocationID': 'int',
    'payment_type': 'int',
    'tip_amount': 'double',
    'fare_amount': 'double'
}


for i in ['01','02','03','04','05','06']:
    taxi_sdf = spark.read.parquet('../data/landing/tlc_data/'+'yellow_tripdata_2018-'+i+'.parquet')

    # since we will not include features like VendorID, Extra and MTA_tax. 
    # we will remove any data that breaks the bussiness rule for these features
    # such we will remove any instance with invalid vendorid, and remove any instance with mta_tax!= 0.5
    taxi_sdf = taxi_sdf.filter(taxi_sdf["VendorID"].isin([1,2]))
    taxi_sdf = taxi_sdf.filter(F.col('MTA_tax') == 0.5)

    # remove any data with invalid Extra any extra other than 0.5 and 1 or 0
    taxi_sdf = taxi_sdf.filter(taxi_sdf["Extra"].isin([0.5,1,0]))

    # remove any data with invalid RateCodeID
    taxi_sdf = taxi_sdf.filter(taxi_sdf["RateCodeID"].isin([1,2,3,4,5,6]))

    # retain any record with fare amount less than total amount
    taxi_sdf = taxi_sdf.filter(taxi_sdf["Fare_amount"] < taxi_sdf["Total_amount"])

    # retain record with valid toll_amount
    taxi_sdf = taxi_sdf.filter(F.col('Tolls_amount') >= 0)
    taxi_sdf= taxi_sdf.filter(F.col('Tolls_amount') <=38)

 
    filter_sdf = taxi_sdf.select('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 
                             'PULocationID', 'DOLocationID', 'payment_type', 'tip_amount', 'Fare_amount')



    for column_name, new_data_type in data_type_mappings.items():
         filter_sdf = filter_sdf.withColumn(column_name, col(column_name).cast(new_data_type))

    consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in filter_sdf.columns]
    filter_sdf = filter_sdf.select(*consistent_col_casing)

    

    output_folder = "../data/raw/taxi_data/" + i  # Replace with the desired output folder
    filter_sdf.write.parquet(output_folder, mode="overwrite")
    

#### Quick exmaine the output in raw data 

In [8]:
taxi_sdf = spark.read.parquet('../data/raw/taxi_data/*')

In [9]:
taxi_sdf

tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pulocationid,dolocationid,payment_type,tip_amount,fare_amount
2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,41,24,2,0.0,4.5
2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,239,140,2,0.0,14.0
2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,262,141,1,1.0,6.0
2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,140,257,2,0.0,33.5
2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,246,239,1,2.75,12.5
2018-01-01 00:29:29,2018-01-01 00:32:48,3,0.5,143,143,2,0.0,4.5
2018-01-01 00:38:08,2018-01-01 00:48:24,2,1.7,50,239,1,2.05,9.0
2018-01-01 00:49:29,2018-01-01 00:51:53,1,0.7,239,238,1,1.0,4.0
2018-01-01 00:56:38,2018-01-01 01:01:05,1,1.0,238,24,1,1.7,5.5
2018-01-01 00:17:04,2018-01-01 00:22:24,1,0.7,170,170,2,0.0,5.5


# Filtering taxi data based on business rules and save in curate data

### Filtering and cleanning

In [10]:
# Iterate through each month, as total 6 month is too big 
for i in ['01','02','03','04','05','06']:
    temp_sdf = spark.read.parquet('../data/raw/taxi_data/' + i)

    # remove any record with invalide passenger counts
    # valid passenger count are between 1-7
    temp_sdf = temp_sdf.where((F.col('passenger_count') > 0) & (F.col('passenger_count')<=7 ))
    
    # remove any record with less than 3 dollar fare_amount and more than $120
    temp_sdf = temp_sdf.filter(F.col('fare_amount') > 3)
    temp_sdf = temp_sdf.filter(F.col('fare_amount') <120)
    
    
    # remove any record with negative tipping and over $100 tip
    temp_sdf = temp_sdf.filter(F.col('tip_amount') >= 0)
    temp_sdf = temp_sdf.filter(F.col('tip_amount') <= 100)
    
    #remove any instance with trip distance less than 0.2 mile 
    temp_sdf = temp_sdf.filter(F.col('trip_distance') >= 0.2)    
    
    #Define valid payment types
    valid_payment_types = [1, 2, 3, 4, 5, 6]
    # Remove rows with invalid payment types
    temp_sdf = temp_sdf.filter(temp_sdf["payment_type"].isin(valid_payment_types))
    
    #Define valid location id
    location_ids = list(range(1,264))
    # Remove rows with invalid pick up location id
    # as we are mostly interest in trips that begin inside new york
    temp_sdf = temp_sdf.filter(temp_sdf["pulocationid"].isin(location_ids))
    
    #filter out records where payment_type is not equal to 1 but tip_amount is greater than 0
    # as record with payment_type!=1 and tip_amount >0 violates the bussiness rule 
    ### remove negative tip
    temp_sdf = temp_sdf.filter((F.col("payment_type") == 1) | (F.col("tip_amount") <= 0))
    
    #filter out all record with non credicard payments 
    temp_sdf = temp_sdf.filter((F.col("payment_type") == 1))
    
    # Create new feature trip_duration to calculate trip duration in minute
    temp_sdf = temp_sdf.withColumn("trip_duration_minutes", round((unix_timestamp(col("tpep_dropoff_datetime")).cast("double") - unix_timestamp(col("tpep_pickup_datetime")).cast("double"))/ 60, 3))
    
    # remove any trip/instance with trip duration less than a minute and longer than 2.5 hours 
    temp_sdf = temp_sdf.filter(F.col('trip_duration_minutes') >= 1)
    temp_sdf = temp_sdf.filter(F.col('trip_duration_minutes') <= 150)

    
    # Create new feature of pickup_day and hour (the assitant feature) from pickup_datetime attribute
    temp_sdf = temp_sdf.withColumn("pickup_day", date_format(col("tpep_pickup_datetime"), "yyyy-MM-dd"))
    temp_sdf = temp_sdf.withColumn("hour", date_format(col("tpep_pickup_datetime"), "HH").cast("int"))
    
    #### Remove any record that is not in our research time period 
    # Define the start and end dates of the desired period
    start_date = datetime.strptime("2018-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2018-06-30", "%Y-%m-%d")
    # Filter out records not within the desired period
    temp_sdf = temp_sdf.filter((col("pickup_day") >= start_date) & (col("pickup_day") <= end_date))

    
    # Extract day of the week from pickup_time (Like Monday, Tuesday.....etc)
    temp_sdf = temp_sdf.withColumn("what_day", date_format(col("pickup_day"), "EEEE"))

    # Include new feature week number
    temp_sdf = temp_sdf.withColumn("week_number", weekofyear("pickup_day"))
    
    # Create a new column for weekday or weekend from what day 
    temp_sdf = temp_sdf.withColumn("day_of_the_week", when(col("what_day").isin("Saturday", "Sunday"), "Weekend").otherwise("Weekday"))
    
    
    # Drop the tpep_pickup_datetime, tpep_dropoff_datetime, and other temporary/assistant features like hour and what_day, since we no longer using them
    temp_sdf = temp_sdf.drop("tpep_pickup_datetime", "tpep_dropoff_datetime", "what_day","payment_type", "dolocationid")
    
    # dropping any row/instance of data contain missing value
    temp_sdf = temp_sdf.na.drop()
    
    output_folder = "../data/curated/taxi_data/" + i  # Replace with the desired output folder
    temp_sdf.write.parquet(output_folder, mode="overwrite")

In [11]:
tlc_sdf = spark.read.parquet('../data/curated/taxi_data/*')
tlc_sdf.count()


35960309

In [12]:
tlc_sdf.orderBy(desc("passenger_count"))

passenger_count,trip_distance,pulocationid,tip_amount,fare_amount,trip_duration_minutes,pickup_day,hour,week_number,day_of_the_week
7,11.79,186,0.0,77.0,23.783,2018-06-28,2,26,Weekday
7,17.18,68,0.0,70.0,29.417,2018-03-04,15,9,Weekend
7,10.13,238,2.0,70.0,15.5,2018-02-10,2,6,Weekend
7,21.16,138,17.41,70.0,32.7,2018-03-06,0,10,Weekday
7,16.02,48,16.26,70.0,24.417,2018-05-08,5,19,Weekday
7,18.62,138,10.0,70.0,24.8,2018-03-14,1,11,Weekday
7,13.47,48,16.66,70.0,26.817,2018-02-14,6,7,Weekday
7,0.24,229,4.96,24.0,4.4,2018-03-25,0,12,Weekend
7,19.87,74,15.16,75.0,30.75,2018-04-06,22,14,Weekday
7,15.73,138,0.0,77.0,39.967,2018-02-20,22,8,Weekday
